# 人力资源规划

## 目标和前提条件

人员配置问题 — 需要对员工的招聘、培训、裁员和排班等做出艰难决策 — 在制造业和服务业中普遍存在。在这个例子中,你将学习如何通过创建一个优化的多期运营计划来建模和求解复杂的人员配置问题,该计划最小化裁员总数和成本。

有关这种类型模型的更多信息,可以在H. Paul Williams的《Model Building in Mathematical Programming》第五版第256-257页和303-304页的例5中找到。

这个建模示例属于高级水平,我们假设你熟悉Python和Gurobi Python API,并且对构建数学优化模型有高级知识。通常,这些示例的目标函数和/或约束条件较为复杂,或需要使用Gurobi Python API的高级功能。

**下载代码库** <br />
你可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。 

---
## 问题描述

一家公司正在改变其业务运营方式,因此其人员配置需求预计会发生变化。

由于购买了新设备,预计将减少对非技术工人的需求,增加对技术工人和半技术工人的需求。此外,由于预计明年将出现经济放缓导致的销售预测下降,预计将进一步减少所有类别的劳动力需求。

未来三年的劳动力需求预测如下:

| <i></i> | 非技术工人 | 半技术工人 | 技术工人 |
| --- | --- | --- | --- |
| 当前人数 | 2000 | 1500 | 1000 |
| 第1年 | 1000 | 1400 | 1000 |
| 第2年 | 500 | 2000 | 1500 |
| 第3年 | 0 | 2500 | 2000 |

公司需要确定未来三年每年的:

- 招聘
- 再培训
- 裁员(裁减)
- 兼职vs全职员工

需要注意的是,劳动力每年都会出现一定程度的自然流失。新员工入职第一年的流失率相对较高,而后续年份的流失率相对较低。预期流失率如下:

| <i></i> | 非技术工人(%) | 半技术工人(%) | 技术工人(%) |
| --- | --- | --- | --- |
| 工作年限< 1年 | 25 | 20 | 10 |
| 工作年限≥ 1年 | 10 | 5 | 5 |

所有现有员工的工作年限都在1年以上。

### 招聘

每年可以从公司外部招聘一定数量的各类员工,具体如下:

| 非技术工人 | 半技术工人 | 技术工人 |
| --- | --- | --- |
| 500 | 800 | 500 |

### 再培训

每年最多可以培训200名非技术工人使其成为半技术工人。每名工人的培训成本为400美元。

此外,也可以培训半技术工人使其成为技术工人。但是,培训人数不能超过当前技术工人队伍的25%,每名工人的培训成本为500美元。

最后,可以将工人降级到较低技能等级。但是,50%的降级工人会离开公司,增加上述的自然流失率。

### 裁员 

每名被裁员工有权获得遣散费,非技术工人200美元,半技术工人或技术工人500美元。

### 超编人员

每年可以有超出实际需求的员工,总数不超过150人,但会产生以下每人每年的额外成本。

| 非技术工人 | 半技术工人 | 技术工人 |
| --- | --- | --- |
| 1500美元 | 2000美元 | 3000美元 |

### 兼职工人

每个技能等级最多可以安排50名员工从事兼职工作。这样做的成本(每名员工,每年)如下:

| 非技术工人 | 半技术工人 | 技术工人 |
| --- | --- | --- |
| 500美元 | 400美元 | 400美元 |

**注意:** 兼职员工的生产力是全职员工的一半。

如果公司的目标是最小化裁员,他们应该采取什么计划?

如果他们的目标是最小化成本,他们可以进一步减少多少成本?

他们如何确定每个工作岗位每年可能的节省额?

---
## 模型公式

### 集合和索引

$t \in \text{Years}=\{1,2,3\}$: 年份集合。

$s \in \text{Skills}=\{s_1: \text{非技术工人},s_2: \text{半技术工人},s_3: \text{技术工人}\}$: 技能等级集合。

### 参数

$\text{rookie_attrition} \in [0,1] \subset \mathbb{R}^+$: 工作年限不满一年的员工离职率。

$\text{veteran_attrition} \in [0,1] \subset \mathbb{R}^+$: 工作年限超过一年的员工离职率。

$\text{demoted_attrition} \in [0,1] \subset \mathbb{R}^+$: 被降级后离开公司的员工比例。

$\text{parttime_cap} \in [0,1] \subset \mathbb{R}^+$: 兼职工人相对于全职工人的生产力。

$\text{max_train_unskilled} \in \mathbb{N}$: 任一年可培训的非技术工人最大数量。

$\text{max_train_semiskilled} \in [0,1] \subset \mathbb{R}^+$: 任一年可培训的半技术工人(相对于技术工人)的最大比例。

$\text{max_parttime} \in \mathbb{N}$: 任一年每个技能等级的兼职工人最大数量。

$\text{max_overmanning} \in \mathbb{N}$: 任一年超编工人的最大数量。

$\text{max_hiring}_s \in \mathbb{N}$: 任一年可招聘的技能等级s的工人最大数量。

$\text{training_cost}_s \in \mathbb{R}^+$: 培训一名技能等级s的工人至下一等级的成本。

$\text{layoff_cost}_s \in \mathbb{R}^+$: 裁退一名技能等级s的工人的成本。

$\text{parttime_cost}_s \in \mathbb{R}^+$: 将一名技能等级s的工人安排为兼职的成本。

$\text{overmanning_cost}_s \in \mathbb{R}^+$: 技能等级s超编人员的年度成本。

$\text{curr_workforce}_s \in \mathbb{N}$: 规划期开始时技能等级s的现有人力。

$\text{demand}_{t,s} \in \mathbb{N}$: t年技能等级s所需的人力。


### 决策变量

$\text{hire}_{t,s} \in [0,\text{max_hiring}_s] \subset \mathbb{R}^+$: t年招聘的技能等级s的工人数量。

$\text{part_time}_{t,s} \in [0,\text{max_parttime}] \subset \mathbb{R}^+$: t年工作的技能等级s的兼职工人数量。

$\text{workforce}_{t,s} \in \mathbb{R}^+$: t年可用的技能等级s的工人数量。

$\text{layoff}_{t,s} \in \mathbb{R}^+$: t年裁退的技能等级s的工人数量。

$\text{excess}_{t,s} \in \mathbb{R}^+$: t年技能等级s的超编工人数量。

$\text{train}_{t,s,s'} \in \mathbb{R}^+$: t年从技能等级s培训到技能等级s'的工人数量。

### 目标函数

- **裁员:** 最小化规划期内的总裁员人数。

\begin{equation}
\text{Minimize} \quad Z = \sum_{t \in \text{Years}}\sum_{s \in \text{Skills}}{\text{layoff}_{t,s}}
\end{equation}

- **成本:** 最小化规划期内培训、超编、兼职工人和裁员产生的总成本(美元)。

\begin{equation}
\text{Minimize} \quad W = \sum_{t \in \text{Years}}{\{\text{training_cost}_{s_1}*\text{train}_{t,s1,s2} + \text{training_cost}_{s_2}*\text{train}_{t,s2,s3}\}}
\end{equation}

\begin{equation}
+ \sum_{t \in \text{Years}}\sum_{s \in \text{Skills}}{\{\text{parttime_cost}*\text{part_time}_{t,s} + \text{layoff_cost}_s*\text{layoff}_{t,s} + \text{overmanning_cost}_s*\text{excess}_{t,s}\}}
\end{equation}

### 约束条件

- **初始平衡:** t=1年可用的技能等级s的劳动力等于上一年的劳动力、新招聘人员、晋升和降级的工人(考虑流失率)、减去裁员和调动的工人。

\begin{equation}
\text{workforce}_{1,s} = (1-\text{veteran_attrition}_s)*\text{curr_workforce} + (1-\text{rookie_attrition}_s)*\text{hire}_{1,s} 
\end{equation}

\begin{equation}
+ \sum_{s' \in \text{Skills} | s' < s}{\{(1-\text{veteran_attrition})*\text{train}_{1,s',s} - \text{train}_{1,s,s'}\}} 
\end{equation}

\begin{equation}
+ \sum_{s' \in \text{Skills} | s' > s}{\{(1-\text{demoted_attrition})*\text{train}_{1,s',s} - \text{train}_{1,s,s'}\}} - \text{layoff}_{1,s} \qquad \forall s \in \text{Skills}
\end{equation}


- **平衡:** t>1年可用的技能等级s的劳动力等于上一年的劳动力、新招聘人员、晋升和降级的工人(考虑流失率)、减去裁员和调动的工人。

\begin{equation}
\text{workforce}_{t,s} = (1-\text{veteran_attrition}_s)*\text{workforce}_{t-1,s} + (1-\text{rookie_attrition}_s)*\text{hire}_{t,s} 
\end{equation}

\begin{equation}
+ \sum_{s' \in \text{Skills} | s' < s}{\{(1-\text{veteran_attrition})*\text{train}_{t,s',s} - \text{train}_{t,s,s'}\}}
\end{equation}

\begin{equation}
+ \sum_{s' \in \text{Skills} | s' > s}{\{(1-\text{demotion_attrition})*\text{train}_{t,s',s} - \text{train}_{t,s,s'}\}} - \text{layoff}_{t,s} \quad \forall (t > 1,s) \in \text{Years} \times \text{Skills}
\end{equation}

- **非技术工人培训:** t年培训的非技术工人不能超过最大限额。非技术工人不能直接培训成技术工人。

\begin{equation}
\text{train}_{t,s_1,s_2} \leq 200 \quad \forall t \in \text{Years}
\end{equation}

\begin{equation}
\text{train}_{t,s_1,s_3} = 0 \quad \forall t \in \text{Years}
\end{equation}

- **半技术工人培训:** t年培训的半技术工人不能超过最大限额。

\begin{equation}
\text{train}_{t,s_2,s_3} \leq 0.25*\text{available}_{t,s_3} \quad \forall t \in \text{Years}
\end{equation}

- **超编:** t年的超编工人不能超过最大限额。

\begin{equation}
\sum_{s \in \text{Skills}}{\text{excess}_{t,s}} \leq \text{max_overmanning} \quad \forall t \in \text{Years}
\end{equation}

- **需求:** t年技能等级s可用的劳动力等于所需工人数加上超编工人和兼职工人。

\begin{equation}
\text{available}_{t,s} = \text{demand}_{t,s} + \text{excess}_{t,s} + \text{parttime_cap}*\text{part_time}_{t,s} \quad \forall (t,s) \in \text{Years} \times \text{Skills}
\end{equation}

---
## Python实现

导入Gurobi Python模块和其他Python库。

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 参数

years = [1, 2, 3]
skills = ['s1', 's2', 's3']

curr_workforce = {'s1': 2000, 's2': 1500, 's3': 1000}
demand = {
    (1, 's1'): 1000,
    (1, 's2'): 1400,
    (1, 's3'): 1000,
    (2, 's1'): 500,
    (2, 's2'): 2000,
    (2, 's3'): 1500,
    (3, 's1'): 0,
    (3, 's2'): 2500,
    (3, 's3'): 2000
}
rookie_attrition = {'s1': 0.25, 's2': 0.20, 's3': 0.10}
veteran_attrition = {'s1': 0.10, 's2': 0.05, 's3': 0.05}
demoted_attrition = 0.50
max_hiring = {
    (1, 's1'): 500,
    (1, 's2'): 800,
    (1, 's3'): 500,
    (2, 's1'): 500,
    (2, 's2'): 800,
    (2, 's3'): 500,
    (3, 's1'): 500,
    (3, 's2'): 800,
    (3, 's3'): 500
}
max_overmanning = 150
max_parttime = 50
parttime_cap = 0.50
max_train_unskilled = 200
max_train_semiskilled = 0.25

training_cost = {'s1': 400, 's2': 500}
layoff_cost = {'s1': 200, 's2': 500, 's3': 500}
parttime_cost = {'s1': 500, 's2': 400, 's3': 400}
overmanning_cost = {'s1': 1500, 's2': 2000, 's3': 3000}

## 模型部署
我们创建一个模型和变量。对于三个技能等级中的每一个以及每一年,我们将创建变量来表示被招聘、转为兼职工作、可用作工人、被裁员或超编的工人数量。对于每对技能等级和每年,我们有一个变量表示被培训到更高/更低技能等级的工人数量。兼职和可招聘的人数是有限的。

In [3]:
manpower = gp.Model('Manpower planning')

hire = manpower.addVars(years, skills, ub=max_hiring, name="Hire")
part_time = manpower.addVars(years, skills, ub=max_parttime,
                          name="Part_time")
workforce = manpower.addVars(years, skills, name="Available")
layoff = manpower.addVars(years, skills, name="Layoff")
excess = manpower.addVars(years, skills, name="Overmanned")
train = manpower.addVars(years, skills, skills, name="Train")

Using license file c:\gurobi\gurobi.lic


接下来,我们插入约束条件。平衡约束确保每个技能等级和每年当前所需的工人(劳动力)以及被裁员的人员,和被培训到当前等级的人员,减去从当前等级培训到不同技能等级的人员,等于上一年的劳动力(或第一年的当前人数)加上招聘的人员。每年都有一定数量的人离开公司,所以这也被视为一个因素。这个约束描述了雇佣员工总数的变化。

In [ ]:
#1.1 & 1.2 平衡

Balance = manpower.addConstrs(
    (workforce[year, level] == (1-veteran_attrition[level])*(curr_workforce[level] if year == 1 else workforce[year-1, level])
    + (1-rookie_attrition[level])*hire[year, level] + gp.quicksum((1- veteran_attrition[level])* train[year, level2, level]
                                                        -train[year, level, level2] for level2 in skills if level2 < level)
    + gp.quicksum((1- demoted_attrition)* train[year, level2, level] -train[year, level, level2] for level2 in skills if level2 > level)
    - layoff[year, level] for year in years for level in skills), "Balance")

非技术工人培训约束强制每年只能有200名工人从非技术工人培训成半技术工人,这是由于能力限制。此外,一年内没有人可以从非技术工人直接培训成技术工人。

In [ ]:
#2.1 & 2.2  非技术培训

UnskilledTrain1 = manpower.addConstrs((train[year, 's1', 's2'] <= max_train_unskilled for year in years), "Unskilled_training1")
UnskilledTrain2 = manpower.addConstrs((train[year, 's1', 's3'] == 0 for year in years), "Unskilled_training2")

半技术工人培训约束规定,半技术工人培训成技术工人的人数不能超过当时技术工人队伍的四分之一。这是由于能力限制。

In [ ]:
#3. 半=技术的培训

SemiskilledTrain = manpower.addConstrs((train[year,'s2', 's3'] <= max_train_semiskilled * workforce[year,'s3'] for year in years), "Semiskilled_training")

超编约束确保一年内所有技能等级的总超编人数不超过150人。

In [ ]:
#4. 超编约束
Overmanning = manpower.addConstrs((excess.sum(year, '*') <= max_overmanning for year in years), "Overmanning")

需求约束确保每个等级和每年的工人数量等于所需工人数量加上超编工人和兼职工作的工人数量。

In [ ]:
#5. 需求
Demand = manpower.addConstrs((workforce[year, level] ==
     demand[year,level] + excess[year, level] + parttime_cap * part_time[year, level]
                     for year in years for level in skills), "Requirements")

第一个目标是最小化被裁员工人的总数。这可以表述为:

In [ ]:
#0.1 目标函数：减少裁员
obj1 = layoff.sum()
manpower.setObjective(obj1, GRB.MINIMIZE)

第二个替代目标是最小化所有雇佣工人和再培训的总成本:

```
obj2 = quicksum((training_cost[level]*train[year, level, skills[skills.index(level)+1]] if level < 's3' else 0)
                + layoff_cost[level]*layoff[year, level]
                + parttime_cost[level]*part_time[year, level]
                + overmanning_cost[level] * excess[year, level] for year in years for level in skills)
```

接下来,我们以最小化裁员的目标函数开始优化,Gurobi找到最优解。

In [10]:
manpower.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 72 columns and 117 nonzeros
Model fingerprint: 0x03e41c91
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 8e+02]
  RHS range        [2e+02, 3e+03]
Presolve removed 18 rows and 44 columns
Presolve time: 0.01s
Presolved: 12 rows, 28 columns, 56 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4000000e+02   6.484375e+01   0.000000e+00      0s
       8    8.4179688e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds
Optimal objective  8.417968750e+02


## 分析

最小裁员人数为841.80。为达到最小裁员人数而采取的最优政策如下。

### 招聘计划
该计划确定在规划期的每一年(行)和每个技能等级(列)要招聘的新工人数量。例如,在第2年,我们将招聘649.3名半技术工人。

In [11]:
rows = years.copy()
columns = skills.copy()
hire_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in hire.keys():
    if (abs(hire[year, level].x) > 1e-6):
        hire_plan.loc[year, level] = np.round(hire[year, level].x, 1)
hire_plan

,s1,s2,s3
1,0.0,0.0,0.0
2,0.0,649.3,500.0
3,0.0,677.0,500.0


### 培训和降级计划
该计划定义了在规划期的每一年要通过培训晋升(或降级)的工人数量。例如,在第1年,我们将把168.4名技术工人(s3)降级为半技术工人(s2)。

In [12]:
rows = years.copy()
columns = ['{0} to {1}'.format(level1, level2) for level1 in skills for level2 in skills if level1 != level2]
train_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level1, level2 in train.keys():
    col = '{0} to {1}'.format(level1, level2)
    if (abs(train[year, level1, level2].x) > 1e-6):
        train_plan.loc[year, col] = np.round(train[year, level1, level2].x, 1)
train_plan

,s1 to s2,s1 to s3,s2 to s1,s2 to s3,s3 to s1,s3 to s2
1,200.0,0.0,0.0,256.2,0.0,168.4
2,200.0,0.0,0.0,80.3,0.0,0.0
3,200.0,0.0,0.0,131.6,0.0,0.0


### 裁员计划

该计划确定在规划期的每一年要裁退的每个技能等级的工人数量。例如,我们将在第3年裁退232.5名非技术工人。

In [13]:
rows = years.copy()
columns = skills.copy()
layoff_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in layoff.keys():
    if (abs(layoff[year, level].x) > 1e-6):
        layoff_plan.loc[year, level] = np.round(layoff[year, level].x, 1)
layoff_plan

,s1,s2,s3
1,443.0,0.0,0.0
2,166.3,0.0,0.0
3,232.5,0.0,0.0


### 兼职计划

该计划定义了在规划期的每一年工作的每个技能等级的兼职工人数量。例如,在第1年,我们有50名兼职技术工人。

In [14]:
rows = years.copy()
columns = skills.copy()
parttime_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in part_time.keys():
    if (abs(part_time[year, level].x) > 1e-6):
        parttime_plan.loc[year, level] = np.round(part_time[year, level].x, 1)
parttime_plan

,s1,s2,s3
1,50.0,50.0,50.0
2,50.0,0.0,0.0
3,50.0,0.0,0.0


### 超编计划

该计划确定了在规划期的每一年每个技能等级的超编工人数量。例如,我们在第3年有150名非技术超编工人。

In [15]:
rows = years.copy()
columns = skills.copy()
excess_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in excess.keys():
    if (abs(excess[year, level].x) > 1e-6):
        excess_plan.loc[year, level] = np.round(excess[year, level].x, 1)
excess_plan

,s1,s2,s3
1,132.0,18.0,0.0
2,150.0,0.0,0.0
3,150.0,0.0,0.0


通过最小化成本,我们可以实施在三年期间总成本为498,677.29美元并导致1,423.7人裁员的政策。可以考虑替代的最优解决方案,以在不增加成本的情况下减少裁员。如果我们最小化成本而不是裁员,我们可以节省942,712.51美元,但代价是增加581.9人的裁员。因此,在最小化裁员时,每保住一个工作岗位的成本可以视为1,620.06美元。

**注意:** 如果你想将解决方案写入文件而不是打印到终端,可以使用model.write()命令。示例实现为:

`manpower.write("manpower-planning-output.sol")`

---
## 参考文献

H. Paul Williams,《Model Building in Mathematical Programming》第五版。

版权所有 &copy; 2020 Gurobi Optimization, LLC